In [112]:
#Import Packages
import numpy as np
import matplotlib.pyplot as plt
from nltk import sent_tokenize
import nltk
import pandas as pd
from nltk.stem import SnowballStemmer
from nltk.stem import WordNetLemmatizer
from nltk.corpus import sentiwordnet as swn
import re
import keras
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [113]:
#Read Dataset
data = pd.read_csv('Train SemEval 2015 Task 12.csv')
test = pd.read_csv('Test SemEval 2015 Task 12.csv')
data1 = pd.read_csv('Train extended.csv')
data1

,sentences/sentence/0/text,sentences/sentence/0/_id,sentences,aspects,polarity,sentences/sentence/1/_id,sentences.1,aspects.1,polarity.1,sentences/sentence/2/_id,...,sentences/sentence/7/Opinions/Opinion/2/_polarity,sentences/sentence/16/_OutOfScope,sentences/sentence/20/_OutOfScope,sentences/sentence/17/_OutOfScope,sentences/sentence/9/Opinions/Opinion/2/_category,sentences/sentence/9/Opinions/Opinion/2/_polarity,sentences/sentence/11/_OutOfScope,sentences/sentence/14/_OutOfScope,sentences/sentence/2/Opinions/Opinion/4/_category,sentences/sentence/2/Opinions/Opinion/4/_polarity
0,Being a PC user my whole life....,79:00:00,This computer is absolutely AMAZING!!!,LAPTOP#GENERAL,positive,79:01:00,10 plus hours of battery...,BATTERY#OPERATION_PERFORMANCE,positive,79:02:00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,the laptop was really good and it goes really ...,10:00,i would really recommend to any person out the...,LAPTOP#GENERAL,positive,10:01,and its really cheap and you wont regret buyin...,LAPTOP#PRICE,positive,10:02,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,"As a lifelong Windows user, I was extremely pl...",264:00:00,"As a computer science student in college, I fi...",LAPTOP#PORTABILITY,positive,264:01:00,without a big ol' clunky machine in my backpac...,LAPTOP#DESIGN_FEATURES,positive,264:02:00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Oh my goodness-I am not a happy camper.,24:00:00,My HP is very heavy.,LAPTOP#DESIGN_FEATURES,negative,24:01:00,Not easy to carry.,LAPTOP#PORTABILITY,negative,24:02:00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,"Since I purchased my Toshiba netbook, I have b...",277:00:00,The netbook is easier for me to take to bed an...,LAPTOP#PORTABILITY,positive,277:01:00,"The screen takes some getting use to, because ...",DISPLAY#USABILITY,neutral,277:02:00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
445,This laptop is amazing!,B00KB3MXH4_22_A106YGESUYA4BP:0,Windows 8.1 has its pros and cons.,OS#GENERAL,neutral,B00KB3MXH4_22_A106YGESUYA4BP:1,The keyboard is backlit but you have to press ...,KEYBOARD#DESIGN_FEATURES,positive,B00KB3MXH4_22_A106YGESUYA4BP:2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
446,The HP Envy 15 has been everything I expected.,B00D7Z84OY_46_A1P70MSK0HHO7K:0,After doing much research for a laptop that ha...,LAPTOP#DESIGN_FEATURES,positive,B00D7Z84OY_46_A1P70MSK0HHO7K:1,Amazon had the best pricing and the delivery w...,NaN,NaN,B00D7Z84OY_46_A1P70MSK0HHO7K:2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
447,used 6 times black screen returned for refund,B00KMRGF28_304_A35Q7MBKQZV1SA:0,do not use the facial recognition,SOFTWARE#GENERAL,negative,B00KMRGF28_304_A35Q7MBKQZV1SA:1,Windows 8 is not supported by a lot of things!,OS#USABILITY,negative,B00KMRGF28_304_A35Q7MBKQZV1SA:2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
448,"The Dell is quick enough, sturdy, not good wit...",B00KMRGF28_143_A2F0D5EV8ZOK44:0,It does what it advertises.,LAPTOP#GENERAL,positive,B00KMRGF28_143_A2F0D5EV8ZOK44:1,"Price is great, wish it didn't have Windows 8,...",LAPTOP#PRICE,positive,B00KMRGF28_143_A2F0D5EV8ZOK44:2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [114]:
#Stemmer and lemmatizer 
snowball_stemmer = SnowballStemmer('english')
wordnet_lemmatizer = WordNetLemmatizer()
stopwords = [ "hmm","ok","th","rd","th","nd","!","#","$","%","&","'","’","u","v","b","r","(",")","*","+","-",".","/",":",";","<","=",">","?","@","[","/","]","^","_","`","{","|","}","~",",","a", "about", "above", "after", "again", "against", "all", "am", "an", "and", "any", "are", "as", "at", "be", "because", "been", "before", "being", "below", "between", "both", "but", "by", "could", "did", "do", "does", "doing", "down", "during", "each", "few", "for", "from", "further", "had", "has", "have", "having", "he", "he'd", "he'll", "he's", "her", "here", "here's", "hers", "herself", "him", "himself", "his", "how", "how's", "i", "i'd", "i'll", "i'm", "i've", "if", "in", "into", "is", "it", "it's", "its", "itself", "let's", "me", "more", "most", "my", "myself", "nor", "of", "on", "once", "only", "or", "other", "ought", "our", "ours", "ourselves", "out", "over", "own", "same", "she","please","sorry","bye", "she'd", "she'll", "she's", "should", "so", "some", "such", "than", "that", "that's", "the", "their", "theirs", "them", "themselves", "then", "there", "there's", "these", "they", "they'd", "they'll", "they're", "they've", "this", "those", "through", "to", "too", "under", "until", "up", "very", "was", "we", "we'd", "we'll", "we're", "we've", "were", "what", "what's", "when", "when's", "where", "where's", "which", "while", "who", "who's", "whom", "why", "why's", "with", "would", "you", "you'd", "you'll", "you're", "you've", "your", "yours", "yourself", "yourselves","hello"]

In [115]:
#Reviews and Aspect Labels
train_sen1 = data.iloc[:,0]
train_sen2 = data.iloc[:,3]
train_sen3 = data.iloc[:,6]
train_sen4 = data1.iloc[:,2]
train_sen5 = data1.iloc[:,6]
test_sen = test.iloc[:,10]
test_aspect = test.iloc[:,11]
aspect_sen1 = data.iloc[:,1]
aspect_sen2 = data.iloc[:,4]
aspect_sen3 = data.iloc[:,7]
aspect_sen4 = data1.iloc[:,3]
aspect_sen5 = data1.iloc[:,7]
aspect_sen = aspect_sen1.append(aspect_sen2)
aspect_sen = aspect_sen.append(aspect_sen3)
aspect_sen = aspect_sen.append(aspect_sen4)
aspect_sen = aspect_sen.append(aspect_sen5)
aspects = aspect_sen.append(test_aspect)
train_sen = train_sen1.append(train_sen2)
train_sen = train_sen.append(train_sen3)
train_sen = train_sen.append(train_sen4)
train_sen = train_sen.append(train_sen5)
train_sen = train_sen.astype(str)
test_sen = test_sen.astype(str)
aspects = aspects.astype(str)
aspects = aspects.astype('category')
aspects = aspects.cat.codes
aspects_train = aspects[:1731]
aspects_test = aspects[1731:]
aspects_train = np.asarray(aspects_train)
aspects_test = np.asarray(aspects_test)

In [116]:
#Sentiment Labels
lab_train1 = data.iloc[:,2]
lab_train2 = data.iloc[:,5]
lab_train3 = data.iloc[:,8]
lab_train4 = data1.iloc[:,4]
lab_train5 = data1.iloc[:,8]
lab_train = lab_train1.append(lab_train2)
lab_train = lab_train.append(lab_train3)
lab_train = lab_train.append(lab_train4)
lab_train = lab_train.append(lab_train5)
lab_test = test.iloc[:,12]

0      0
1      3
2      0
3      1
4      3
      ..
168    3
169    0
170    1
171    3
172    0
Length: 1904, dtype: int8

In [119]:
#Data Structure
sen=[]
for x in train_sen:
    #Sentence Tokenization
    sentence = sent_tokenize(x)
    
    for line in sentence:
        
        #Convert to lowercase
        line=line.lower()
        #Remove Digits
        line = ''.join(c for c in line if not c.isdigit())
        #Word Tokenization
        token= nltk.word_tokenize(line)
        #Stopword Removal
        token=[words for words in token if words not in stopwords]
        #Lemmatization
        token = [wordnet_lemmatizer.lemmatize(word) for word in token]
        #Stemming
        token= [snowball_stemmer.stem(word) for word in token]
        #POS Tagging
        tags = nltk.pos_tag(token)

    sen.append(str(tags))
    
sen = pd.DataFrame(sen)
sen = sen.rename(columns={0: "a"})
print(sen)

                                                      a
0     [('pc', 'NN'), ('user', 'NN'), ('whole', 'JJ')...
1     [('laptop', 'JJ'), ('realli', 'NN'), ('good', ...
2     [('lifelong', 'RB'), ('window', 'WRB'), ('user...
3     [('oh', 'RB'), ('goodness-i', 'NN'), ('not', '...
4     [('sinc', 'NN'), ('purchas', 'NN'), ('toshiba'...
...                                                 ...
1726  [('keyboard', 'NN'), ('backlit', 'NN'), ('pres...
1727  [('amazon', 'RB'), ('best', 'JJS'), ('price', ...
1728  [('window', 'NN'), ('not', 'RB'), ('support', ...
1729  [('price', 'NN'), ('great', 'JJ'), ('wish', 'J...
1730                                    [('nan', 'NN')]

[1731 rows x 1 columns]


In [121]:
#Data Structure
sent=[]
c=0
for x in test_sen:
    sentence = sent_tokenize(x)
    
    for line in sentence:
        
        #Convert to lowercase
        line=line.lower()
        #Remove Digits
        line = ''.join(c for c in line if not c.isdigit())
        #Word Tokenization
        token= nltk.word_tokenize(line)
        #Stopword Removal
        token=[words for words in token if words not in stopwords]
        #Lemmatization
        token = [wordnet_lemmatizer.lemmatize(word) for word in token]
        #Stemming
        token= [snowball_stemmer.stem(word) for word in token]
        #POS Tagging
        tags = nltk.pos_tag(token)
        
    sent.append(str(tags))
sent = pd.DataFrame(sent)
sent = sent.rename(columns={0: "a1"})
print(sent)

                                                    a1
0    [('lock', 'NN'), ('search', 'NN'), ('updat', '...
1    [('buy', 'VB'), ('high', 'JJ'), ('recommend', ...
2                                      [('nan', 'NN')]
3    [('solid-st', 'JJ'), ('memori', 'NNS'), ('make...
4                                      [('nan', 'NN')]
..                                                 ...
168  [('keyboard', 'NN'), ('backlit', 'NN'), ('pres...
169  [('amazon', 'RB'), ('best', 'JJS'), ('price', ...
170  [('window', 'NN'), ('not', 'RB'), ('support', ...
171  [('price', 'NN'), ('great', 'JJ'), ('wish', 'J...
172                                    [('nan', 'NN')]

[173 rows x 1 columns]


In [123]:
#Feature Engineering and Encoding 
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer()
sen_vectors = vectorizer.fit_transform(sen['a'])
sent_vectors = vectorizer.transform(sent['a1'])
print(train_lab)

  (1, 471)	0.6472129509118695
  (1, 678)	0.7623092523195228
  (2, 893)	1.0
  (3, 377)	1.0
  (6, 365)	1.0
  (7, 289)	0.6207264067732229
  (7, 365)	0.6815764588194205
  (7, 471)	0.3874948499239469
  (8, 291)	1.0
  (10, 365)	0.6966923313535964
  (10, 471)	0.39608863670062877
  (10, 758)	0.5981083407792673
  (11, 25)	0.8405102880310843
  (11, 471)	0.5417955848047338
  (12, 364)	0.42823655891112844
  (12, 365)	0.9036666695258576
  (13, 318)	1.0
  (15, 34)	1.0
  (16, 743)	1.0
  (17, 365)	1.0
  (21, 365)	1.0
  (26, 622)	1.0
  (27, 471)	1.0
  (28, 471)	0.41800337835063267
  (28, 723)	0.6968413458159658
  :	:
  (1694, 471)	0.44872394545868
  (1694, 689)	0.5604085360040267
  (1696, 297)	0.7670700853885739
  (1696, 901)	0.6415633126215727
  (1701, 562)	0.6928711067496339
  (1701, 698)	0.7210614602317457
  (1702, 540)	1.0
  (1704, 459)	1.0
  (1705, 862)	1.0
  (1707, 471)	1.0
  (1710, 324)	0.7071067811865475
  (1710, 586)	0.7071067811865475
  (1712, 365)	1.0
  (1716, 365)	0.7662169785040125
  (1716

In [130]:
#Model SVM with Time Frames for Sentiment Classification 
import time
from sklearn import svm
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score

classifier_linear = svm.SVC(kernel='linear')
t0 = time.time()
classifier_linear.fit(sen_vectors, lab_train)
t1 = time.time()
prediction_linear = classifier_linear.predict(sent_vectors)
t2 = time.time()
time_linear_train = t1-t0
time_linear_predict = t2-t1

#Results
print("Training time: %fs; Prediction time: %fs" % (time_linear_train, time_linear_predict))
report = accuracy_score(lab_test, prediction_linear)
report1 = classification_report(lab_test, prediction_linear, output_dict=True)
print(report1)
print(report)

Training time: 1.687101s; Prediction time: 0.046865s
{'nan': {'precision': 0.9692307692307692, 'recall': 0.8513513513513513, 'f1-score': 0.9064748201438849, 'support': 74}, 'negative': {'precision': 0.8478260869565217, 'recall': 0.9069767441860465, 'f1-score': 0.8764044943820224, 'support': 43}, 'neutral': {'precision': 1.0, 'recall': 0.3333333333333333, 'f1-score': 0.5, 'support': 3}, 'positive': {'precision': 0.8360655737704918, 'recall': 0.9622641509433962, 'f1-score': 0.8947368421052632, 'support': 53}, 'accuracy': 0.8901734104046243, 'macro avg': {'precision': 0.9132806074894457, 'recall': 0.7634813949535318, 'f1-score': 0.7944040391577927, 'support': 173}, 'weighted avg': {'precision': 0.8987923356765516, 'recall': 0.8901734104046243, 'f1-score': 0.888355968674297, 'support': 173}}
0.8901734104046243


In [129]:
#Model SVM with Time Frames for Aspect Categorization 
import time
from sklearn import svm
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score

classifier_linear = svm.SVC(kernel='linear')
t0 = time.time()
classifier_linear.fit(sen_vectors, aspects_train)
t1 = time.time()
prediction_linear = classifier_linear.predict(sent_vectors)
t2 = time.time()
time_linear_train = t1-t0
time_linear_predict = t2-t1

#Results
print("Training time: %fs; Prediction time: %fs" % (time_linear_train, time_linear_predict))
report = accuracy_score(aspects_test, prediction_linear)
report1 = classification_report(aspects_test, prediction_linear, output_dict=True)
print(report1)
print(report)

Training time: 4.984656s; Prediction time: 0.249940s


C:\Users\hp\Anaconda3\envs\PythonGPU\lib\site-packages\sklearn\metrics\classification.py:1437: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


{'0': {'precision': 0.0, 'recall': 0.0, 'f1-score': 0.0, 'support': 1}, '1': {'precision': 0.0, 'recall': 0.0, 'f1-score': 0.0, 'support': 1}, '2': {'precision': 0.75, 'recall': 1.0, 'f1-score': 0.8571428571428571, 'support': 3}, '4': {'precision': 1.0, 'recall': 0.5, 'f1-score': 0.6666666666666666, 'support': 2}, '6': {'precision': 0.0, 'recall': 0.0, 'f1-score': 0.0, 'support': 1}, '7': {'precision': 1.0, 'recall': 1.0, 'f1-score': 1.0, 'support': 1}, '8': {'precision': 1.0, 'recall': 0.6666666666666666, 'f1-score': 0.8, 'support': 3}, '10': {'precision': 1.0, 'recall': 1.0, 'f1-score': 1.0, 'support': 2}, '17': {'precision': 1.0, 'recall': 1.0, 'f1-score': 1.0, 'support': 3}, '18': {'precision': 1.0, 'recall': 1.0, 'f1-score': 1.0, 'support': 2}, '21': {'precision': 1.0, 'recall': 0.5, 'f1-score': 0.6666666666666666, 'support': 4}, '22': {'precision': 0.7142857142857143, 'recall': 1.0, 'f1-score': 0.8333333333333333, 'support': 10}, '23': {'precision': 0.75, 'recall': 0.882352941176